In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm_notebook as tqdm

In [9]:
training_initial = pd.read_csv('../data/hima/all_400_features_3rdsept.csv')
training_initial.drop(['common_neighbor'], axis=1, inplace=True)
training_initial.head()

,Source,Sink,AAprediction,exist_in_training_dataset,common_neighbours,JA
0,1709040,1695026,3.669171,1,"{4575620, 3709961, 3503374, 4332049, 4761107, ...",0.001105
1,20388,4700562,0.262151,1,"{3553485, 2726331, 3550837}",0.000004
2,4067692,2495732,0.000000,1,set(),0.000000
3,2912811,345595,0.000000,1,set(),0.000000
4,3077895,3951788,0.421203,1,"{3637241, 3376555, 3149884}",0.002562


In [10]:
temp = list(training_initial.to_records(index=False))

In [11]:
common_neighbors=[]
with tqdm(total=len(temp)) as pbar:
    for source, sink, AAprediction, exist, neighbors, JAprediction in temp:
        if neighbors=='set()':
            temp_list = []
            common_neighbors.append((source, sink, exist, AAprediction, JAprediction, temp_list, len(temp_list)))
        else:
            neighbors_string = (neighbors[1:len(neighbors)-1]).split(sep=',')
            neighbors_array = [int(x) for x in neighbors_string]
            common_neighbors.append((source, sink, exist, AAprediction, JAprediction, neighbors_array, len(neighbors_array)))
        pbar.update(1)

In [12]:
training_with_cnb = pd.DataFrame.from_records(common_neighbors, columns=['source','sink','exist_in_training_dataset', 'AAprediction', 'JAprediction','common_neighbors','num_cnb'])
training_with_cnb.head()

,source,sink,exist_in_training_dataset,AAprediction,JAprediction,common_neighbors,num_cnb
0,1709040,1695026,1,3.669171,0.001105,"[4575620, 3709961, 3503374, 4332049, 4761107, ...",39
1,20388,4700562,1,0.262151,0.000004,"[3553485, 2726331, 3550837]",3
2,4067692,2495732,1,0.000000,0.000000,[],0
3,2912811,345595,1,0.000000,0.000000,[],0
4,3077895,3951788,1,0.421203,0.002562,"[3637241, 3376555, 3149884]",3


Storing it in a file: Common Neighbors

In [13]:
training_with_cnb.to_csv('../data/40k_TRAIN_class3features.csv', index=False)

#### Generating `class1features` for Training Dataset

In [14]:
node_details = pd.read_csv('../data/node_features_2Sep18.csv')
node_details.head()

,Node,in_degree,out_degree,degree
0,4066935,142,143,285
1,1272125,37,20,57
2,3105725,76,79,155
3,2828522,28,26,54
4,4394015,107,222,329


In [17]:
class1features = pd.merge(training_with_cnb, node_details, left_on='source', right_on='Node', how='inner', left_index=False, right_index=False)
class1features.drop(['Node','degree'], axis=1, inplace=True)
class1features.rename({'in_degree':'source_in_degree','out_degree':'source_out_degree'}, axis=1, inplace=True)
class1features.head()

,source,sink,exist_in_training_dataset,AAprediction,JAprediction,common_neighbors,num_cnb,source_in_degree,source_out_degree
0,1709040,1695026,1,3.669171,0.001105,"[4575620, 3709961, 3503374, 4332049, 4761107, ...",39,640,35279
1,1709040,4631595,1,0.371067,0.000113,"[3465568, 2984603, 4306404, 3267663]",4,640,35279
2,1709040,2293501,1,2.183257,0.000680,"[2776192, 410900, 759828, 2903457, 2757027, 42...",24,640,35279
3,1709040,3082315,1,3.543801,0.001077,"[3133826, 3503374, 3563279, 4204052, 506389, 1...",38,640,35279
4,1709040,560617,1,11.818681,0.003282,"[3671045, 2610697, 4042761, 4332049, 3507219, ...",116,640,35279


In [18]:
class1features = pd.merge(class1features, node_details, left_on='sink', right_on='Node', how='inner', left_index=False, right_index=False)
class1features.drop(['Node','degree'], axis=1, inplace=True)
class1features.rename({'in_degree':'sink_in_degree','out_degree':'sink_out_degree'}, axis=1, inplace=True)
class1features.head()

,source,sink,exist_in_training_dataset,AAprediction,JAprediction,common_neighbors,num_cnb,source_in_degree,source_out_degree,sink_in_degree,sink_out_degree
0,1709040,1695026,1,3.669171,0.001105,"[4575620, 3709961, 3503374, 4332049, 4761107, ...",39,640,35279,52,0
1,4204052,1695026,1,3.651442,0.000417,"[4575620, 3709961, 4135692, 3503374, 3926802, ...",39,1010,93387,52,0
2,3709961,1695026,1,2.630250,0.004939,"[3503374, 3926802, 4761107, 4204052, 2984603, ...",29,332,5749,52,0
3,1709040,4631595,1,0.371067,0.000113,"[3465568, 2984603, 4306404, 3267663]",4,640,35279,7,0
4,1709040,2293501,1,2.183257,0.000680,"[2776192, 410900, 759828, 2903457, 2757027, 42...",24,640,35279,31,0


In [28]:
class1features['source_outBYin_ratio'] = class1features['source_out_degree']/class1features['source_in_degree']
class1features['source_inBYout_ratio'] = class1features['source_in_degree']/class1features['source_out_degree']
class1features['sink_outBYin_ratio'] = class1features['sink_out_degree']/class1features['sink_in_degree']
class1features['sink_inBYout_ratio'] = class1features['sink_in_degree']/class1features['sink_out_degree']
class1features['source_outBYin_ratio'].replace(np.inf, 0, inplace=True)
class1features['source_inBYout_ratio'].replace(np.inf, 0, inplace=True)
class1features['sink_outBYin_ratio'].replace(np.inf, 0, inplace=True)
class1features['sink_inBYout_ratio'].replace(np.inf, 0, inplace=True)
class1features.to_csv('../data/40k_TRAIN_class1features.csv', index=False)
class1features.head()

,source,sink,exist_in_training_dataset,AAprediction,JAprediction,common_neighbors,num_cnb,source_in_degree,source_out_degree,sink_in_degree,sink_out_degree,source_outBYin_ratio,source_inBYout_ratio,sink_outBYin_ratio,sink_inBYout_ratio
0,1709040,1695026,1,3.669171,0.001105,"[4575620, 3709961, 3503374, 4332049, 4761107, ...",39,640,35279,52,0,55.123438,0.018141,0.0,0.0
1,4204052,1695026,1,3.651442,0.000417,"[4575620, 3709961, 4135692, 3503374, 3926802, ...",39,1010,93387,52,0,92.462376,0.010815,0.0,0.0
2,3709961,1695026,1,2.630250,0.004939,"[3503374, 3926802, 4761107, 4204052, 2984603, ...",29,332,5749,52,0,17.316265,0.057749,0.0,0.0
3,1709040,4631595,1,0.371067,0.000113,"[3465568, 2984603, 4306404, 3267663]",4,640,35279,7,0,55.123438,0.018141,0.0,0.0
4,1709040,2293501,1,2.183257,0.000680,"[2776192, 410900, 759828, 2903457, 2757027, 42...",24,640,35279,31,0,55.123438,0.018141,0.0,0.0


#### Reading the training dataset to create comparision set: `string` and using that information to calculate `class2features`

In [29]:
train_df = pd.read_csv('../data/df.csv')
edgeStringSet = set(train_df['string'].unique())

In [42]:
class2features = []
with tqdm(total=len(common_neighbors)) as pbar:
    for source,sink,exist,AAprediction,JAprediction,neighbors,num_cnb in common_neighbors:
        t1 = t2 = t3 = t4 = 0
        for item in neighbors:
            source2neighbor = str(source) + ',' + str(item)
            sink2neighbor = str(sink) + ',' + str(item)
            neighbor2source = str(item) + ',' + str(source)
            neighbor2sink = str(item) + ',' + str(sink)
            if source2neighbor in edgeStringSet and neighbor2source in edgeStringSet:
                t1 = t1 + 1
            if source2neighbor in edgeStringSet and sink2neighbor in edgeStringSet:
                t2 = t2 + 1
            if neighbor2source in edgeStringSet and neighbor2sink in edgeStringSet:
                t3 = t3 + 1
            if neighbor2source in edgeStringSet and sink2neighbor in edgeStringSet:
                t4 = t4 + 1
        # print(t1, t2, t3, t4)
        if num_cnb == 0:
            class2features.append((source, sink, neighbors, num_cnb, t1, t2, t3, t4, 0, 0, 0, 0, exist))
        else:
            class2features.append((source, sink, neighbors, num_cnb, t1, t2, t3, t4, t1/num_cnb, t2/num_cnb, t3/num_cnb, t4/num_cnb, exist))
        pbar.update(1)

In [44]:
class2features_df = pd.DataFrame.from_records(class2features, columns=['source', 'sink', 'common_neighbors', 'num_cnb', 't1', 't2', 't3', 't4', 't1BYnum_cnb', 't2BYnum_cnb', 't3BYnum_cnb', 't4BYnum_cnb','exist_in_training_dataset'])
class2features_df['exist_in_training_dataset'].head()

195700

In [50]:
train_all_features = pd.merge(class2features_df, class1features, how='inner', left_on=['source','sink'], right_on=['source','sink'], left_index=False, right_index=False)
train_all_features.drop(['common_neighbors_y','num_cnb_y','exist_in_training_dataset_y'], axis=1, inplace=True)
train_all_features.rename({'common_neighbors_x':'common_neighbors','num_cnb_x':'num_cnb','exist_in_training_dataset_x':'exist_in_training_dataset'},axis=1, inplace=True)
train_all_features.head()

,source,sink,common_neighbors,num_cnb,t1,t2,t3,t4,t1BYnum_cnb,t2BYnum_cnb,...,AAprediction,JAprediction,source_in_degree,source_out_degree,sink_in_degree,sink_out_degree,source_outBYin_ratio,source_inBYout_ratio,sink_outBYin_ratio,sink_inBYout_ratio
0,1709040,1695026,"[4575620, 3709961, 3503374, 4332049, 4761107, ...",39,38,0,38,0,0.974359,0.0,...,3.669171,0.001105,640,35279,52,0,55.123438,0.018141,0.0,0.0
1,20388,4700562,"[3553485, 2726331, 3550837]",3,3,0,3,0,1.000000,0.0,...,0.262151,0.000004,4841,761793,4,0,157.362735,0.006355,0.0,0.0
2,4067692,2495732,[],0,0,0,0,0,0.000000,0.0,...,0.000000,0.000000,109,10458,1,0,95.944954,0.010423,0.0,0.0
3,2912811,345595,[],0,0,0,0,0,0.000000,0.0,...,0.000000,0.000000,83,4829,3,0,58.180723,0.017188,0.0,0.0
4,3077895,3951788,"[3637241, 3376555, 3149884]",3,3,0,3,0,1.000000,0.0,...,0.421203,0.002562,355,1149,5,0,3.236620,0.308964,0.0,0.0


Saving features to file

In [54]:
print(train_all_features.shape)
train_all_features.columns

(391400, 23)


Index(['source', 'sink', 'common_neighbors', 'num_cnb', 't1', 't2', 't3', 't4',
       't1BYnum_cnb', 't2BYnum_cnb', 't3BYnum_cnb', 't4BYnum_cnb',
       'exist_in_training_dataset', 'source_in_degree', 'source_out_degree',
       'sink_in_degree', 'sink_out_degree', 'source_outBYin_ratio',
       'source_inBYout_ratio', 'sink_outBYin_ratio', 'sink_inBYout_ratio',
       'AAprediction', 'JAprediction'],
      dtype='object')

Rearranging

In [53]:
train_all_features=train_all_features[['source', 'sink', 'common_neighbors', 'num_cnb', 't1', 't2', 't3', 't4',
       't1BYnum_cnb', 't2BYnum_cnb', 't3BYnum_cnb', 't4BYnum_cnb', 'exist_in_training_dataset',
       'source_in_degree', 'source_out_degree', 'sink_in_degree',
       'sink_out_degree', 'source_outBYin_ratio', 'source_inBYout_ratio',
       'sink_outBYin_ratio', 'sink_inBYout_ratio', 'AAprediction',
       'JAprediction']]

In [55]:
train_all_features.head()

,source,sink,common_neighbors,num_cnb,t1,t2,t3,t4,t1BYnum_cnb,t2BYnum_cnb,...,source_in_degree,source_out_degree,sink_in_degree,sink_out_degree,source_outBYin_ratio,source_inBYout_ratio,sink_outBYin_ratio,sink_inBYout_ratio,AAprediction,JAprediction
0,1709040,1695026,"[4575620, 3709961, 3503374, 4332049, 4761107, ...",39,38,0,38,0,0.974359,0.0,...,640,35279,52,0,55.123438,0.018141,0.0,0.0,3.669171,0.001105
1,20388,4700562,"[3553485, 2726331, 3550837]",3,3,0,3,0,1.000000,0.0,...,4841,761793,4,0,157.362735,0.006355,0.0,0.0,0.262151,0.000004
2,4067692,2495732,[],0,0,0,0,0,0.000000,0.0,...,109,10458,1,0,95.944954,0.010423,0.0,0.0,0.000000,0.000000
3,2912811,345595,[],0,0,0,0,0,0.000000,0.0,...,83,4829,3,0,58.180723,0.017188,0.0,0.0,0.000000,0.000000
4,3077895,3951788,"[3637241, 3376555, 3149884]",3,3,0,3,0,1.000000,0.0,...,355,1149,5,0,3.236620,0.308964,0.0,0.0,0.421203,0.002562


In [56]:
train_all_features.to_csv('../data/40k_TRAIN_All_features.csv', index=False)